*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [1]:
# import libraries
#try:
  # %tensorflow_version only exists in Colab.
  #!pip install tf-nightly
#except Exception:
  #pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
#!pip install tensorflow-datasets
#import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.8.2


In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

--2022-08-30 19:44:41--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.008s  

2022-08-30 19:44:41 (43.8 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2022-08-30 19:44:41--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

In [3]:
train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [4]:
train_data = pd.read_table(train_file_path, names=['label', 'sms'])
train_sms = train_data['sms']
train_labels = train_data['label']
test_data = pd.read_table(test_file_path, names=['label', 'sms'])
test_sms = test_data['sms']
test_labels = test_data['label']

In [5]:
train_sms = [sms.split() for sms in train_sms]
test_sms = [sms.split() for sms in test_sms]

In [6]:
train_labels = np.array(list(map(lambda x: 1 if x == 'spam' else 0, train_labels)))
test_labels = np.array(list(map(lambda x: 1 if x == 'spam' else 0, test_labels)))

In [7]:
vocab = []
for sms in train_sms:
  words = np.unique(sms)
  for word in words:
    vocab.append(word)
vocab = np.unique(np.array(vocab))

In [8]:
str_to_int = keras.layers.StringLookup(vocabulary=vocab)
int_list = []
for sms in train_sms:
  int_list.append(str_to_int(sms))
train_sms = int_list
int_list = []
for sms in test_sms:
  int_list.append(str_to_int(sms))
test_sms = int_list

In [9]:
MAXLEN = 200
train_sms = keras.preprocessing.sequence.pad_sequences(train_sms, MAXLEN)
test_sms = keras.preprocessing.sequence.pad_sequences(test_sms, MAXLEN)

In [10]:
model = keras.Sequential([
    keras.layers.Embedding(len(vocab)+1, 64),
    keras.layers.LSTM(64),
    keras.layers.Dense(1, activation='sigmoid')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          725184    
                                                                 
 lstm (LSTM)                 (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 758,273
Trainable params: 758,273
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

In [12]:
history = model.fit(train_sms, train_labels, epochs=10)

Epoch 1/10
131/131 [==============================] - 9s 11ms/step - loss: 0.1881 - acc: 0.9347
Epoch 2/10
131/131 [==============================] - 1s 10ms/step - loss: 0.0488 - acc: 0.9852
Epoch 3/10
131/131 [==============================] - 1s 10ms/step - loss: 0.0312 - acc: 0.9911
Epoch 4/10
131/131 [==============================] - 1s 10ms/step - loss: 0.0207 - acc: 0.9943
Epoch 5/10
131/131 [==============================] - 1s 10ms/step - loss: 0.0157 - acc: 0.9964
Epoch 6/10
131/131 [==============================] - 1s 10ms/step - loss: 0.0107 - acc: 0.9976
Epoch 7/10
131/131 [==============================] - 1s 10ms/step - loss: 0.0051 - acc: 0.9988
Epoch 8/10
131/131 [==============================] - 1s 11ms/step - loss: 0.0030 - acc: 0.9995
Epoch 9/10
131/131 [==============================] - 1s 11ms/step - loss: 0.0024 - acc: 0.9993
Epoch 10/10
131/131 [==============================] - 1s 10ms/step - loss: 0.0019 - acc: 0.9995


In [13]:
result = model.evaluate(test_sms, test_labels)
result

44/44 [==============================] - 1s 5ms/step - loss: 0.1040 - acc: 0.9820


[0.10403535515069962, 0.9820402264595032]

In [14]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(text):
  text = text.split()
  text = str_to_int(text)
  text = keras.preprocessing.sequence.pad_sequences([text], MAXLEN)
  model_pred = model.predict(text)
  probability = model_pred[0][0]
  label = 'spam' if probability >= 0.5 else 'ham'
  prediction = [probability, label]
  return prediction

pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)

[3.1771888e-06, 'ham']


In [15]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!
